## Quality Measures

In [2]:
## Display in Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
from faker import Faker
import sqlite3
import random

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'
import warnings
warnings.filterwarnings('ignore')


# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
Faker.seed(42)
random.seed(42)

In [5]:
from spellchecker import SpellChecker
import pandas as pd

# Initialize SpellChecker
spell = SpellChecker()

def compute_column_metrics(df):
    """
    Compute column-specific quality metrics for a single dataset.

    Args:
        df (pd.DataFrame): The dataset to analyze.

    Returns:
        dict: A dictionary of DataFrames with metrics for each column.
    """
    column_metrics = {}

    for column in df.columns:
        metrics = {}

        # Completeness (Non-missing values)
        metrics['Completeness'] = 100 - df[column].isnull().mean() * 100

        # Validity Checks
        if column == 'Email':
            email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
            metrics['Validity'] = df[column].str.match(email_regex, na=False).mean() * 100

        elif column == 'Salary':
            salary_range = (20000, 2000000)
            metrics['Validity'] = ((df[column] >= salary_range[0]) & (df[column] <= salary_range[1])).mean() * 100

        elif column == 'Birthdate':
            metrics['Validity'] = pd.to_datetime(df[column], errors='coerce').notna().mean() * 100

        elif column == 'SocialSecurity':
            ssn_regex = r'^\d{3}-\d{2}-\d{4}$'
            metrics['Validity'] = df[column].str.match(ssn_regex, na=False).mean() * 100

        # Uniqueness
        if column in ['Name', 'Email', 'SocialSecurity']:
            metrics['Uniqueness'] = df[column].nunique() / len(df) * 100

        # Text-Specific Metrics
        if column in ['Name', 'JobTitle', 'JobDescription']:
            # Capitalization Consistency
            metrics['Capitalization Consistency'] = df[column].str.match(r'^[A-Z][^A-Z]*$', na=False).mean() * 100

            # Special Characters Presence
            special_chars = r'[!@#$%^&*()]'
            metrics['Special Characters'] = 100 - df[column].str.contains(special_chars, na=False).mean() * 100

            # Spelling Accuracy
            def check_spelling(text):
                if isinstance(text, str):
                    words = text.split()
                    misspelled = spell.unknown(words)
                    return len(misspelled) == 0
                return True

            metrics['Spelling Accuracy'] = df[column].apply(check_spelling).mean() * 100

            # Whitespace Consistency
            metrics['Whitespace Consistency'] = df[column].str.match(r'^[^\s].*[^\s]$', na=False).mean() * 100

            # Text Length Validity
            min_len, max_len = 3, 50
            metrics['Text Length Validity'] = df[column].apply(lambda x: min_len <= len(str(x)) <= max_len).mean() * 100

        # Store metrics for the column
        column_metrics[column] = pd.DataFrame(metrics, index=[column]).transpose()

    return column_metrics

# Read in the dataset
employee_data = pd.read_csv('employee_messy_data.csv')

# Compute column-specific metrics
column_specific_metrics = compute_column_metrics(employee_data)

# Display each column's metrics as a separate table
for column, metrics_table in column_specific_metrics.items():
    print(f"\nMetrics for column: {column}")
    print(metrics_table)



Metrics for column: Name
                                  Name
Completeness                100.000000
Uniqueness                   90.914875
Capitalization Consistency    0.025449
Special Characters           50.820715
Spelling Accuracy             2.443059
Whitespace Consistency      100.000000
Text Length Validity        100.000000

Metrics for column: HighestDegree
              HighestDegree
Completeness      88.993511

Metrics for column: Division
              Division
Completeness  88.75175

Metrics for column: DivisionAddress
              DivisionAddress
Completeness        89.375239

Metrics for column: PersonalAddress
              PersonalAddress
Completeness        88.777198

Metrics for column: Email
                  Email
Completeness  89.082580
Validity      55.274208
Uniqueness    80.926327

Metrics for column: PhoneNumber
              PhoneNumber
Completeness    88.459091

Metrics for column: Department
              Department
Completeness   89.477033

Metrics fo